In [1]:
import pandas as pd
from sqlalchemy import create_engine
import yaml
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
config = yaml.safe_load(open('dart_config.yml'))
db_url = 'mysql://' + config['DB_USER'] + ':' + config['DB_USER_PASSWORD'] + '@' + config['DB_HOST'] \
                            + '/' + config['DB_NAME'] + '?charset=utf8'
engine = create_engine(db_url, encoding='utf-8')
conn = engine.connect()
company_list = pd.read_sql_table(config['DB_TABLE_NAME_1'], con=conn,
                          columns=['name'])['name']
company_list = company_list.drop_duplicates().reset_index(drop=True)
company_list.head()

0    롯데케미칼
1     대한유화
2     이화산업
3    디케이앤디
4     송원산업
Name: name, dtype: object

In [3]:
# Read csv
trade_df = pd.read_csv("./data/trade.csv", encoding="utf-8", parse_dates=['ISSUEDT'],
                       dtype={'HSCD': str, "QTY": float, 'BYRADDR2': str, 'SPLYADDR2': str})
trade_df.head()

,HSCD,QTY,QTYUNIT,AMT,CUR,USDUNTPRICE,UNTPRICE,ITEM,BYRORGNM1,BYRPRTNUM,...,BYRADDR2,SPLYORGNM1,SPLYPRTNUM,SPLYADDR1,SPLYADDR2,ISSUEDT,TOTQTY,TOTQTYUNIT,TOTAMT,TOTAMTCUR
0,7210491090,84000.0,KG,45360.00,USD,540.00,540.00,GI COIL,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
1,7210491090,1463000.0,KG,821355.00,USD,585.00,585.00,GI COIL,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
2,7210491090,600000.0,KG,398580.00,USD,664.30,664.30,HG COIL - POSCO,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
3,7225509000,2398000.0,KG,416301.78,USD,501.15,501.15,CR COIL (0.5MM<=T<1.0MM),주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
4,7209169000,23100000.0,KG,8134262.00,USD,490.00,490.00,CR COIL (3MM<=T),주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD


In [4]:
buyer_list = trade_df['BYRORGNM1']
buyer_list.head()

0    주식회사 포스코대우
1    주식회사 포스코대우
2    주식회사 포스코대우
3    주식회사 포스코대우
4    주식회사 포스코대우
Name: BYRORGNM1, dtype: object

In [5]:
supplier_list = trade_df['SPLYORGNM1']
supplier_list.head()

0    POSCO
1    POSCO
2    POSCO
3    POSCO
4    POSCO
Name: SPLYORGNM1, dtype: object

In [6]:
merge_list = buyer_list.append(supplier_list, ignore_index=True).drop_duplicates().reset_index(drop=True)
merge_list.head()

0         주식회사 포스코대우
1        도레이케미칼 주식회사
2    도레이첨단소재주식회사 4공장
3        도레이첨단소재주식회사
4    도레이첨단소재주식회사 2공장
dtype: object

In [7]:
parsed_merge_list = merge_list.str.replace("주식회사", "").str.replace("\(주\)", "")\
        .str.replace("\(유\)", "").str.replace("㈜", "").str.replace(" ", "")
parsed_merge_list.head()

0         포스코대우
1        도레이케미칼
2    도레이첨단소재4공장
3       도레이첨단소재
4    도레이첨단소재2공장
dtype: object

In [8]:
original_merge_list = company_list.append(merge_list, ignore_index=True)
parsed_merge_list = company_list.append(merge_list, ignore_index=True)
del merge_list
print('original_merge_list length:', len(original_merge_list), 'parsed_merge_list length:', len(parsed_merge_list))

original_merge_list length: 54100 parsed_merge_list length: 54100


In [9]:
# make tfidf matrix set
tfidf = TfidfVectorizer(analyzer='char', ngram_range=(2,3))
tfidf_matrix = tfidf.fit_transform(parsed_merge_list)
tfidf_matrix.shape

(54100, 107373)

In [10]:
print('total company count:', len(company_list))

total company count: 796


## Make dictionary

In [16]:
company_dict = {}

In [29]:
# Set company for comparing
company_i += 1
cosine_sim = linear_kernel(tfidf_matrix[company_i], tfidf_matrix)
# print(company_i, original_merge_list[company_i])

# Result comparing
sorted_index = sorted(range(len(cosine_sim[0])), key=lambda i: cosine_sim[0][i])[-10:]
sorted_index.reverse()
for i, v in enumerate(sorted_index):
    print(str(i), str(v), original_merge_list[v], str(cosine_sim[0][v]))
#    if (cosine_sim[0][v] < 0.4):
#        break

0 14400 유니온머티리얼 1.0
1 14 유니온머티리얼 1.0
2 17649 유니온머티리얼(주) 0.8419613030534953
3 17964 유니온머티리얼(주)포항공장 0.6888801362612768
4 1089 유니온 0.46490358036504414
5 438 유니온 0.46490358036504414
6 50118 원창머티리얼 0.39636076897691663
7 41385 일진머티리얼즈 0.37711966315515577
8 5 일진머티리얼즈 0.37711966315515577
9 2585 센텀머티리얼 0.37677120730860414


In [15]:
sim_list = []
for i, v in enumerate(sorted_index):
    sim_list.append(original_merge_list[v])
    if (i == 4):
        break
for v, i in enumerate(sim_list):
    print(v, i)

0 대한유화
1 대한유화
2 대한유화(주)
3 대한유화주식회사
4 대한유화공업(주)


In [13]:
company_dict[original_merge_list[company_i]] = sim_list
print(original_merge_list[company_i], "added to dictionay !!!")
# company_dict

NameError: name 'company_dict' is not defined

### Warning!!

In [3241]:
sim_list.pop(1)
for v, i in enumerate(sim_list):
    print(v, i)

0 엔에스
1 (주)엔에스


## Save

In [3269]:
import json
with open('data/company.json', 'w') as f:
    json.dump(company_dict, f)

## Open

In [240]:
import json
with open('data/company.json') as f:
    my_dict = json.load(f)
my_dict

{'국도화학': ['국도화학', '국도화학(주)', '국도화학주식회사', '국도화학 주식회사', '국도화학(주) 익산지점'],
 '금호석유화학': ['금호석유화학',
  '금호석유화학',
  '금호석유화',
  '금호석유화학(주)',
  '금호석유화학㈜',
  '주금호석유화학',
  '금호석유화학주식회사',
  '금호석유화학 (주)',
  '금호석유(주)',
  '금호석유화학(주)울산고무공장',
  '금호석유화학(주)여수정밀화학공장',
  '금호석유화학(주)여수고무제1공장'],
 '대주전자재료': ['대주전자재료', '대주전자재료', '대주전자재료(주)', '대주전자재료주식회사'],
 '대한유화': ['대한유화', '대한유화', '대한유화(주)', '대한유화주식회사', '대한유화공업(주)', '대한유압'],
 '디케이앤디': ['디케이앤디', '디케이앤디', '(주)디케이앤디', '주식회사 디케이앤디'],
 '롯데케미칼': ['롯데케미칼',
  '롯데케미칼',
  '롯데케미칼(주)',
  '롯데케미칼주식회사',
  '롯데케미칼 (주)',
  '롯데케미칼 주식회사',
  '(주)롯데케미칼',
  '롯데케미컬(주)',
  '롯데케미컬 주식회사'],
 '멜파스': ['멜파스', '주)멜파스', '(주)멜파스', '주식회사 멜파스'],
 '미원상사': ['미원상사', '미원상사(주)', '미원상사 주식회사', '미원상사 (주)', '미원상사㈜', '(주)미원상사'],
 '송원산업': ['송원산업', '송원산업', '송원산업(주)', '송원산업주식회사', '송원산업 (주)', '송원산업 주식회사'],
 '이화산업': ['이화산업', '이화산업', '이화산업사', '이화산업(주)'],
 '일진머티리얼즈': ['일진머티리얼즈',
  '일진머티리얼즈',
  '일진머티리얼즈(주)',
  '일진머티리얼즈 주식회사',
  '일진머티리얼즈 (주)',
  '일진머티리얼즈(주)익산공장'],
 '한국쉘석유': ['한국쉘석유',
  '한국쉘석유',
  '한국쉘석유주식회사',
  '한국쉘석

## dict processing

In [3262]:
company_dict

{'AP위성': ['AP위성', 'AP위성(주)'],
 'CJ씨푸드': ['CJ씨푸드', 'CJ씨푸드(주)', 'CJ씨푸드(주) 이천공장', 'CJ씨푸드(주) 성남공장'],
 'CMG제약': ['CMG제약', '주식회사 CMG제약', '(주)CMG제약', '주식회사CMG제약', '(주)CMG제약 경기지점'],
 'DMS': ['DMS', 'DMS', '(주)DMS'],
 'E1': ['E1', '주식회사 E1'],
 'EG': ['EG', '주식회사 EG', '(주)EG', '주식회사EG'],
 'GH신소재': ['GH신소재', 'GH신소재(주)'],
 'GS글로벌': ['GS글로벌', '(주)GS글로벌'],
 'JW홀딩스': ['JW홀딩스', 'JW홀딩스'],
 'KC코트렐': ['KC코트렐', '(주)KC코트렐', 'KC코트렐(주)', 'KC 코트렐 (주)'],
 'KG케미칼': ['KG케미칼', 'KG케미칼'],
 'KH바텍': ['KH바텍', 'KH바텍', '(주)KH바텍'],
 'KJ프리텍': ['KJ프리텍', 'KJ프리텍', '(주)KJ프리텍'],
 'KPX케미칼': ['KPX케미칼',
  'KPX케미칼',
  'KPX케미칼(주)',
  'KPX케미칼주식회사',
  'KPX케미칼 주식회사',
  'KPX케미칼(주))',
  'kpx케미칼(주)울산공장',
  'KPX케미칼(주)울산공장',
  'KPX케미칼(주) 울산공장',
  'KPX케미칼 (주) 울산공장',
  'KPX케미칼㈜ 천안공장',
  'KPX 케미칼',
  'KPX 케미칼 주식회사',
  'KPX 케미칼(주) 울산공장',
  'KPX 케미칼 (주) 울산공장',
  'KPX CHEMICAL CO.,LTD'],
 'S&TC': ['S&TC', '(주)S&TC', '(주) S&TC', '주식회사 S&TC'],
 'S&T중공업': ['S&T중공업', 'S&T중공업(주)', 'S&T중공업주식회사'],
 'SK네트웍스': ['SK네트웍스', 'SK네트웍스(주)', 'SK네트웍스 (주)'],
 'SPC

In [3267]:
for arr in company_dict:
    company_dict[arr] = list(set(company_dict[arr]))

In [3268]:
company_dict

{'AP위성': ['AP위성', 'AP위성(주)'],
 'CJ씨푸드': ['CJ씨푸드(주) 이천공장', 'CJ씨푸드(주) 성남공장', 'CJ씨푸드(주)', 'CJ씨푸드'],
 'CMG제약': ['(주)CMG제약', '주식회사CMG제약', '주식회사 CMG제약', '(주)CMG제약 경기지점', 'CMG제약'],
 'DMS': ['(주)DMS', 'DMS'],
 'E1': ['주식회사 E1', 'E1'],
 'EG': ['주식회사EG', '(주)EG', '주식회사 EG', 'EG'],
 'GH신소재': ['GH신소재', 'GH신소재(주)'],
 'GS글로벌': ['GS글로벌', '(주)GS글로벌'],
 'JW홀딩스': ['JW홀딩스'],
 'KC코트렐': ['KC코트렐(주)', '(주)KC코트렐', 'KC코트렐', 'KC 코트렐 (주)'],
 'KG케미칼': ['KG케미칼'],
 'KH바텍': ['(주)KH바텍', 'KH바텍'],
 'KJ프리텍': ['(주)KJ프리텍', 'KJ프리텍'],
 'KPX케미칼': ['KPX케미칼(주))',
  'KPX케미칼',
  'KPX케미칼 (주) 울산공장',
  'KPX 케미칼(주) 울산공장',
  'KPX 케미칼 주식회사',
  'KPX케미칼㈜ 천안공장',
  'KPX 케미칼',
  'KPX케미칼(주)울산공장',
  'KPX CHEMICAL CO.,LTD',
  'KPX케미칼(주) 울산공장',
  'KPX케미칼 주식회사',
  'KPX 케미칼 (주) 울산공장',
  'KPX케미칼(주)',
  'KPX케미칼주식회사',
  'kpx케미칼(주)울산공장'],
 'S&TC': ['주식회사 S&TC', '(주) S&TC', '(주)S&TC', 'S&TC'],
 'S&T중공업': ['S&T중공업(주)', 'S&T중공업주식회사', 'S&T중공업'],
 'SK네트웍스': ['SK네트웍스(주)', 'SK네트웍스', 'SK네트웍스 (주)'],
 'SPC삼립': ['(주)SPC삼립', 'SPC삼립'],
 'STX': ['주식회사 STX', 'STX'

## MAKE DF

In [3332]:
for arr in company_dict:
    for word in company_dict[arr]:
        print(arr, word)
        try:
            trade_df.loc[trade_df['BYRORGNM1'] == str(word), 'BYRORGNM1'] = trade_df.loc[trade_df['BYRORGNM1'] == str(word), 'BYRORGNM1'].str.replace(str(word), str(arr))
            trade_df.loc[trade_df['SPLYORGNM1'] == str(word), 'SPLYORGNM1'] = trade_df.loc[trade_df['SPLYORGNM1'] == str(word), 'SPLYORGNM1'].str.replace(str(word), str(arr))
        except:
            continue

남해화학 남해화학(주) 여수공장
남해화학 남해화학(주)여수공장
남해화학 남해화학
남해화학 남해화학(주)
케이엔더블유 (주)케이엔더블유
케이엔더블유 주식회사 케이엔더블유
케이엔더블유 케이엔더블유
빅텍 빅텍
빅텍 (주)빅텍
램테크놀러지 램테크놀러지(주)
램테크놀러지 램테크놀러지
램테크놀러지 램테크놀러지주식회사
두산인프라코어 두산인프라코어 (주)
두산인프라코어 두산인프라코어(주)
두산인프라코어 두산인프라코어주식회사
두산인프라코어 두산인프라코어
두산인프라코어 두산인프라코아(주)
원풍 원풍
원풍 (주) 원풍
원풍 주 원풍
원풍 (주)원풍
제노텍 제노텍
제노텍 (주)제노텍
에너토크 에너토크
에너토크 주식회사 에너토크
유티아이 주식회사 유티아이
유티아이 유티아이
대명코퍼레이션 (주) 대명코퍼레이션
대명코퍼레이션 대명코퍼레이션
네오팜 네오팜
네오팜 주식회사 네오팜
클리오 (주)클리오
클리오 클리오
클리오 (주) 클리오
화인베스틸 (주)화인베스틸
화인베스틸 화인베스틸
인베니아 인베니아 주식회사
인베니아 인베니아
유성기업 유성기업
유성기업 유성기계
유성기업 유성기업(주)
오킨스전자 주식회사 오킨스전자
오킨스전자 (주)오킨스전자
오킨스전자 오킨스전자
레이 레이코퍼레이션
레이 레이
수산중공업 (주) 수산중공업
수산중공업 수산중공업
수산중공업 (주)수산중공업
인콘 (주)인콘
인콘 주식회사 인콘
인콘 인콘
대성엘텍 (주) 대성엘텍
대성엘텍 (주)대성엘텍
대성엘텍 대성엘텍
에스마크 주식회사 에스마크
에스마크 에스마크
인성정보 (주)인성정보
인성정보 인성정보
코웰패션 코웰패션
코웰패션 코웰패션주식회사
코웰패션 코웰패션 주식회사
코웰패션 코웰패션 (주)
코웰패션 코웰패션(주)
평화산업 평화산업사
평화산업 (주)평화산업
평화산업 평화산업㈜
평화산업 평화산업
평화산업 주식회사 평화산업
평화산업 평화산업(주)
삼보판지 삼보판지
삼보판지 (주)삼보판지
삼보판지 주식회사 삼보판지
삼보판지 주식회사삼보판지
퍼스텍 퍼스텍
퍼스텍 퍼스텍(주)
광동제약 광동제약
광동제약 광동제약주식회사
광동제약 광동제약(

한화케미칼 한화케미칼 (주) 여수
한화케미칼 한화케미칼 (주) 울산 2
한화케미칼 한화케미칼
한화케미칼 한화케미컬주식회사
한화케미칼 한화케미칼 (주)여수
한화케미칼 한화케미칼(주)티디아이
한화케미칼 한화케미칼㈜여수
한화케미칼 한화케미칼(주)울산2
한화케미칼 한화케미칼(주)울산1
한화케미칼 한화케미칼 (주) 울산2
한화케미칼 한화케미칼(주)여수공장
한화케미칼 한화케미칼(주) 울산3
한화케미칼 한화케미칼(주) 티디아이
한화케미칼 한화케미칼(주) 폴리실리콘
한화케미칼 한화케미칼(주) 울산, 1
한화케미칼 한화케미칼(주) 울산
한화케미칼 한화케미칼(주)
리노공업 리노공업
리노공업 리노공업 (주)
리노공업 리노공업(주)서울지점
리노공업 리노공업(주)
원일특강 (주)원일특강
원일특강 원일특강
원일특강 (주) 원일특강
케이피에스 주식회사 케이피에스
케이피에스 케이피에스
넥센 주식회사 넥센
넥센 넥센
넥센 (주) 넥센
일진머티리얼즈 일진머티리얼즈(주)익산공장
일진머티리얼즈 일진머티리얼즈
일진머티리얼즈 일진머티리얼즈(주)
일진머티리얼즈 일진머티리얼즈 주식회사
일진머티리얼즈 일진머티리얼즈 (주)
율촌화학 율촌화학주식회사
율촌화학 율촌화학(주)
율촌화학 (주)율촌화학
율촌화학 율촌화학
세원물산 세원물산
세원물산 (주)세원물산
세원물산 주식회사 세원물산
세원셀론텍 세원셀론텍(주)
세원셀론텍 세원셀론텍(주) 성수지점
세원셀론텍 주식회사 세원셀론텍
세원셀론텍 세원셀론텍
씨앗 (주)씨앗
씨앗 (주) 씨앗
씨앗 씨앗
씨앗 주식회사 씨앗
대정화금 대정화금주
대정화금 대정화금(주)
대정화금 대정화금
대정화금 (주)대정화금
일동제약 일동제약(주) 
일동제약 일동제약(주) 안성공장
일동제약 일동제약
일동제약 일동제약(주)안성공장
일동제약 일동제약(주)
티에스이 티에스이
티에스이 (주) 티에스이
티에스이 (주)티에스이
대유에이피 주식회사 대유에이피
대유에이피 (주)대유에이피
대유에이피 대유에이피
대한광통신 대한광통신
대한광통신 대한광통신주식회사
대한광통신 대한광통신 (주)
대한광통신 대한광통신(주

KPX케미칼 KPX 케미칼(주) 울산공장
KPX케미칼 KPX 케미칼 주식회사
KPX케미칼 KPX케미칼㈜ 천안공장
KPX케미칼 KPX 케미칼
KPX케미칼 KPX케미칼(주)울산공장
KPX케미칼 KPX CHEMICAL CO.,LTD
KPX케미칼 KPX케미칼(주) 울산공장
KPX케미칼 KPX케미칼 주식회사
KPX케미칼 KPX 케미칼 (주) 울산공장
KPX케미칼 KPX케미칼(주)
KPX케미칼 KPX케미칼주식회사
KPX케미칼 kpx케미칼(주)울산공장
에스케이씨에스 (주)에스케이씨에스
에스케이씨에스 주식회사 에스케이씨에스
에스케이씨에스 주식회사에스케이씨에스
에스케이씨에스 에스케이씨에스
아이케이세미콘 (주)아이케이세미콘
아이케이세미콘 아이케이세미콘(주)
아이케이세미콘 아이케이세미콘
루트로닉 루트로닉
루트로닉 (주)루트로닉
명문제약 명문제약(주)
명문제약 명문제약 (주)
명문제약 명문제약
휴스틸 휴스틸(주)
휴스틸 (주)휴스틸
휴스틸 주식회사 휴스틸
휴스틸 휴스틸
휴스틸 주식회사휴스틸
아진산업 아진산업(주)
아진산업 아진산업
아진산업 (주)아진산업
아진산업 주식회사 아진산업
하나마이크론 하나마이크론(주)
하나마이크론 주식회사 하나마이크론
하나마이크론 하나마이크론
대덕전자 대덕전자
대덕전자 대덕전자 (주)
대덕전자 대덕전자 주식회사
대덕전자 대덕전자(주)
삼보모터스 삼보모터스 주식회사
삼보모터스 삼보모터스(주)
삼보모터스 삼보모터스㈜
삼보모터스 삼보모터스
삼보모터스 삼보모터스주식회사
디에이테크놀로지 디에이테크놀로지
디에이테크놀로지 (주)디에이테크놀로지
광림 광림
광림 (주)광림
광림 주식회사 광림
켐트로스 (주)켐트로스
켐트로스 켐트로스
켐트로스 (주)켐트로스 안산공장
유비쿼스 주식회사 유비쿼스
유비쿼스 유비쿼스
이수페타시스 이수페타시스
이수페타시스 주식회사이수페타시스
이수페타시스 (주)이수페타시스
평화정공 평화정공㈜
평화정공 평화정공
평화정공 ㈜ 평화정공
평화정공 평화정공(주)본사
평화정공 평화정공(주) 본사
평화정공 평화정공(주)
조선내화 조선내화(주)
조선내화 조선내화(주

대한유화 대한유화공업(주)
대한유화 대한유화
써니전자 써니전자
써니전자 써니전자(주)
써니전자 써니전자㈜
JW홀딩스 JW홀딩스
롯데정밀화학 롯데정밀화학(주)
롯데정밀화학 롯데정밀화학주식회사
롯데정밀화학 롯데정밀화학 (주)
롯데정밀화학 롯데정밀화학
롯데정밀화학 롯데정밀화학주
피엔에이치테크 피엔에이치테크
피엔에이치테크 (주)피엔에이치테크
뷰웍스 (주)뷰웍스
뷰웍스 뷰웍스
한성기업 한성기업(주) 서울영업소
한성기업 한성기업(주)
한성기업 한성기업(주)서울영업소
한성기업 한성기업
세방전지 세방전지(주)
세방전지 세방전지
세방전지 세방전지 (주)
시그네틱스 시그네틱스
시그네틱스 시그네틱스(주)
남선알미늄 (주)남선알미늄
남선알미늄 남선알미늄
남선알미늄 주)남선알미늄
남선알미늄 (주) 남선알미늄
DMS (주)DMS
DMS DMS
제이스텍 제이스텍
제이스텍 (주)제이스텍
덕양산업 덕양산업
덕양산업 덕양산업(주)
덕양산업 덕양산업㈜
하이스틸 주식회사하이스틸
하이스틸 하이스틸
동일제강 동일제강 (주)
동일제강 동일제강
동일제강 동일제강(주)
금강철강 금강철강주식회사
금강철강 금강철강
금강철강 금강철강(주)
이엘케이 이엘케이(주)
이엘케이 에이엘케이
이엘케이 이엘케이
경인전자 경인전자
경인전자 주식회사 경인전자
루미마이크로 루미마이크로(주)
루미마이크로 루미마이크로
루미마이크로 루미마이크로 주식회사
대동공업 대동공업(주)
대동공업 대동공업
유에스티 (주)유에스티
유에스티 유에스티
에이스테크 (주)에이스테크놀로지
에이스테크 주)에이스테크놀로지 서
에이스테크 에이스테크놀로지
에이스테크 에이스테크
에이스테크 에이스테크주식회사
에이스테크 주식회사 에이스테크
에이스테크 (주)에이스테크놀로지 서울지점
에이블씨엔씨 (주) 에이블씨엔씨
에이블씨엔씨 에이블씨엔씨
에이블씨엔씨 (주)에이블씨엔씨
에이블씨엔씨 에이블씨앤씨
씨티네트웍스 (주)씨티네트웍스
씨티네트웍스 씨티네트웍스
케이디켐 케이디켐(주)
케이디켐 (주)케이디켐
케이디켐 케이디켐
휴메딕스 (주) 휴메딕스
휴메딕스 휴메딕스
휴메딕스 (주)휴

화승인더스트리 (주)화승인더스트리아산공장
화승인더스트리 (주)화승인더스트리 아산공장
화승인더스트리 (주)화승인터스트리
화승인더스트리 (주)화승인더스트리
화승인더스트리 (주) 화승인더스트리아산공장
화승인더스트리 화승인더스트리
현대오토에버 현대오토에버주식회사 
현대오토에버 현대오토에버
우리이앤엘 우리이엔엘
우리이앤엘 우리이앤엘
우리이앤엘 우리이앤엘(주)
동성화학 (주) 동성화학
동성화학 (주)동성화학
동성화학 동성화학
동성화학 주식회사 동성화학
해성디에스 해성디에스 주식회사
해성디에스 해성디에스(주)
해성디에스 주식회사 해성디에스
해성디에스 해성디에스
서울전자통신 서울전자통신(주)
서울전자통신 서울전자통신
럭스피아 럭스피아
럭스피아 럭스피아(주)
와이오엠 주식회사 와이오엠
와이오엠 와이오엠
이니텍 이니텍
이니텍 (주)이니텍
극동유화 극동유화주식회사
극동유화 극동유화 (주)
극동유화 극동유화(주)
극동유화 극동유화 주식회사
극동유화 극동유화㈜
극동유화 극동유화
상신이디피 상신이디피(주)
상신이디피 상신이디피주천안지점
상신이디피 상신이디피주식회사
상신이디피 상신이디피
알에프텍 (주)알에프텍
알에프텍 알에프텍
천보 (주)천보
천보 천보
천보 주식회사천보
천보 주식회사 천보
세동 세동
세동 (주)세동
세동 주식회사 세동
대동고려삼 대동고려삼(주)
대동고려삼 대동고려삼 (주)
대동고려삼 대동고려삼
이원컴포텍 이원컴포텍(주)
이원컴포텍 이원컴포텍
네이처셀 (주)네이처셀
네이처셀 네이처셀
금호석유화학 금호석유화학(주)여수정밀화학공장
금호석유화학 금호석유화학(주)여수고무제1공장
금호석유화학 금호석유화학주식회사
금호석유화학 주금호석유화학
금호석유화학 금호석유화학(주)
금호석유화학 금호석유화
금호석유화학 금호석유화학
금호석유화학 금호석유화학 (주)
금호석유화학 금호석유화학(주)울산고무공장
금호석유화학 금호석유(주)
금호석유화학 금호석유화학㈜
국순당 (주) 국순당
국순당 국순당 횡성공장
국순당 국순당 고창명주
국순당 국순당
국순당 (주)국순당
국순당 농업회사법인 국순당 고창명주(주)

In [3324]:
company_dict

{'AP위성': ['AP위성', 'AP위성(주)'],
 'CJ씨푸드': ['CJ씨푸드(주) 이천공장', 'CJ씨푸드(주) 성남공장', 'CJ씨푸드(주)', 'CJ씨푸드'],
 'CMG제약': ['(주)CMG제약', '주식회사CMG제약', '주식회사 CMG제약', '(주)CMG제약 경기지점', 'CMG제약'],
 'DMS': ['(주)DMS', 'DMS'],
 'E1': ['주식회사 E1', 'E1'],
 'EG': ['주식회사EG', '(주)EG', '주식회사 EG', 'EG'],
 'GH신소재': ['GH신소재', 'GH신소재(주)'],
 'GS글로벌': ['GS글로벌', '(주)GS글로벌'],
 'JW홀딩스': ['JW홀딩스'],
 'KC코트렐': ['KC코트렐(주)', '(주)KC코트렐', 'KC코트렐', 'KC 코트렐 (주)'],
 'KG케미칼': ['KG케미칼'],
 'KH바텍': ['(주)KH바텍', 'KH바텍'],
 'KJ프리텍': ['(주)KJ프리텍', 'KJ프리텍'],
 'KPX케미칼': ['KPX케미칼(주))',
  'KPX케미칼',
  'KPX케미칼 (주) 울산공장',
  'KPX 케미칼(주) 울산공장',
  'KPX 케미칼 주식회사',
  'KPX케미칼㈜ 천안공장',
  'KPX 케미칼',
  'KPX케미칼(주)울산공장',
  'KPX CHEMICAL CO.,LTD',
  'KPX케미칼(주) 울산공장',
  'KPX케미칼 주식회사',
  'KPX 케미칼 (주) 울산공장',
  'KPX케미칼(주)',
  'KPX케미칼주식회사',
  'kpx케미칼(주)울산공장'],
 'S&TC': ['주식회사 S&TC', '(주) S&TC', '(주)S&TC', 'S&TC'],
 'S&T중공업': ['S&T중공업(주)', 'S&T중공업주식회사', 'S&T중공업'],
 'SK네트웍스': ['SK네트웍스(주)', 'SK네트웍스', 'SK네트웍스 (주)'],
 'SPC삼립': ['(주)SPC삼립', 'SPC삼립'],
 'STX': ['주식회사 STX', 'STX'

In [3333]:
trade_df

,HSCD,QTY,QTYUNIT,AMT,CUR,USDUNTPRICE,UNTPRICE,ITEM,BYRORGNM1,BYRPRTNUM,...,BYRADDR2,SPLYORGNM1,SPLYPRTNUM,SPLYADDR1,SPLYADDR2,ISSUEDT,TOTQTY,TOTQTYUNIT,TOTAMT,TOTAMTCUR
0,7210491090,84000.0,KG,4.536000e+04,USD,540.00,540.00,GI COIL,NaN,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
1,7210491090,1463000.0,KG,8.213550e+05,USD,585.00,585.00,GI COIL,NaN,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
2,7210491090,600000.0,KG,3.985800e+05,USD,664.30,664.30,HG COIL - POSCO,NaN,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
3,7225509000,2398000.0,KG,4.163018e+05,USD,501.15,501.15,CR COIL (0.5MM<=T<1.0MM),NaN,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
4,7209169000,23100000.0,KG,8.134262e+06,USD,490.00,490.00,CR COIL (3MM<=T),NaN,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
5,7210491090,165000.0,KG,8.580000e+04,USD,520.00,520.00,GI COIL,NaN,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
6,7225509000,5500000.0,KG,3.587000e+05,USD,412.00,412.00,CR COIL (0.5MM<=T<1.0MM),NaN,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
7,7210491090,242000.0,KG,1.558480e+05,USD,644.00,644.00,GI COIL,NaN,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
8,7210491090,222000.0,KG,1.508046e+05,USD,679.30,679.30,HG COIL - POSCO,NaN,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
9,7210491090,144000.0,KG,7.315200e+04,USD,508.00,508.00,GI COIL,NaN,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD


In [3301]:
trade_df_columns = trade_df.columns.tolist()
new_df = pd.DataFrame(columns=trade_df_columns)
new_df.head()

,HSCD,QTY,QTYUNIT,AMT,CUR,USDUNTPRICE,UNTPRICE,ITEM,BYRORGNM1,BYRPRTNUM,...,BYRADDR2,SPLYORGNM1,SPLYPRTNUM,SPLYADDR1,SPLYADDR2,ISSUEDT,TOTQTY,TOTQTYUNIT,TOTAMT,TOTAMTCUR


In [3318]:
trade_df.loc[trade_df['BYRORGNM1'] == '포스코대우테스트', 'BYRORGNM1'] = trade_df.loc[trade_df['BYRORGNM1'] == '포스코대우테스트', 'BYRORGNM1'].str.replace('포스코대우테스트', '주식회사 포스코대우')

In [3331]:
trade_df.loc[trade_df['SPLYORGNM1'] == '남해화학']

,HSCD,QTY,QTYUNIT,AMT,CUR,USDUNTPRICE,UNTPRICE,ITEM,BYRORGNM1,BYRPRTNUM,...,BYRADDR2,SPLYORGNM1,SPLYPRTNUM,SPLYADDR1,SPLYADDR2,ISSUEDT,TOTQTY,TOTQTYUNIT,TOTAMT,TOTAMTCUR


In [3303]:
trade_df.loc[trade_df['BYRORGNM1'] == '주식회사 포스코대우']

,HSCD,QTY,QTYUNIT,AMT,CUR,USDUNTPRICE,UNTPRICE,ITEM,BYRORGNM1,BYRPRTNUM,...,BYRADDR2,SPLYORGNM1,SPLYPRTNUM,SPLYADDR1,SPLYADDR2,ISSUEDT,TOTQTY,TOTQTYUNIT,TOTAMT,TOTAMTCUR
0,7210491090,84000.0,KG,45360.00,USD,540.00,540.00,GI COIL,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
1,7210491090,1463000.0,KG,821355.00,USD,585.00,585.00,GI COIL,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
2,7210491090,600000.0,KG,398580.00,USD,664.30,664.30,HG COIL - POSCO,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
3,7225509000,2398000.0,KG,416301.78,USD,501.15,501.15,CR COIL (0.5MM<=T<1.0MM),주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
4,7209169000,23100000.0,KG,8134262.00,USD,490.00,490.00,CR COIL (3MM<=T),주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
5,7210491090,165000.0,KG,85800.00,USD,520.00,520.00,GI COIL,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
6,7225509000,5500000.0,KG,358700.00,USD,412.00,412.00,CR COIL (0.5MM<=T<1.0MM),주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
7,7210491090,242000.0,KG,155848.00,USD,644.00,644.00,GI COIL,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
8,7210491090,222000.0,KG,150804.60,USD,679.30,679.30,HG COIL - POSCO,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
9,7210491090,144000.0,KG,73152.00,USD,508.00,508.00,GI COIL,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD


In [3270]:
trade_df

,HSCD,QTY,QTYUNIT,AMT,CUR,USDUNTPRICE,UNTPRICE,ITEM,BYRORGNM1,BYRPRTNUM,...,BYRADDR2,SPLYORGNM1,SPLYPRTNUM,SPLYADDR1,SPLYADDR2,ISSUEDT,TOTQTY,TOTQTYUNIT,TOTAMT,TOTAMTCUR
0,7210491090,84000.0,KG,4.536000e+04,USD,540.00,540.00,GI COIL,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
1,7210491090,1463000.0,KG,8.213550e+05,USD,585.00,585.00,GI COIL,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
2,7210491090,600000.0,KG,3.985800e+05,USD,664.30,664.30,HG COIL - POSCO,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
3,7225509000,2398000.0,KG,4.163018e+05,USD,501.15,501.15,CR COIL (0.5MM<=T<1.0MM),주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
4,7209169000,23100000.0,KG,8.134262e+06,USD,490.00,490.00,CR COIL (3MM<=T),주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
5,7210491090,165000.0,KG,8.580000e+04,USD,520.00,520.00,GI COIL,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
6,7225509000,5500000.0,KG,3.587000e+05,USD,412.00,412.00,CR COIL (0.5MM<=T<1.0MM),주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
7,7210491090,242000.0,KG,1.558480e+05,USD,644.00,644.00,GI COIL,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
8,7210491090,222000.0,KG,1.508046e+05,USD,679.30,679.30,HG COIL - POSCO,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD
9,7210491090,144000.0,KG,7.315200e+04,USD,508.00,508.00,GI COIL,주식회사 포스코대우,1318555320,...,주식회사 포스코대우,POSCO,5068100017,"1 GOEDONG-DONG, NAM-GU",NaN,2018-02-27,565475000.0,KG,2.577329e+08,USD


## TEST

In [126]:
trade_df.loc[trade_df['BYRORGNM1'] == '현대자동차']['HSCD'].drop_duplicates()

792170     8708999000
792177     8302300000
792209     7318220000
792510     9031909000
792518     8512201090
792519     3923500000
792520     8708991030
792548     8543709090
792553     8708309000
792593     9030339000
792725     8708400000
792849     8301200000
792857     8302100000
792925     8501101000
792931     8501311010
792934     8708290000
792957     3926300000
792962     8301600000
793240     7326909000
793271     4009310000
793282     4009420000
793284     8708930000
793295     4016930000
793299     4009110000
793322     4016991090
793326     8415900000
793386     4009120000
793440     4009410000
793498     9401902000
793755     8536502000
              ...    
889182     9027509000
889261     8536490000
892477     3921901000
892507     4415100000
892619     4415200000
892932     8207909000
894509     3402901000
894510     3402903000
895094     8414901000
895207     8708502000
895654     8512202090
896409     8431100000
903947     3918109000
904261     3209101019
904266    

In [127]:
trade_df.loc[trade_df['HSCD'] == '3923500000']['BYRORGNM1'].drop_duplicates()

47242              주식회사 포스코대우
53902              엘지이노텍 주식회사
132944             (주)지앤지트레이딩
331914                제라 주식회사
336422                  (주)사람
342035                (주)네트코스
342062               유앤팩 주식회사
342191                 (주)파세코
344238                  (주)신영
346942                건식무역(주)
360620                (주)세신산업
370437             한국후지제록스(주)
372635              (주)신흥문구공업
384526              고려인삼과학(주)
461201                  기아자동차
699674             현대다이모스주식회사
699873               한국스미토모상사
705920           (주)에이테크오토모티브
722031           뉴팩코퍼레이션 유한회사
725462               한국지엠주식회사
772986             (주)트라이던트상사
780586                 (주)코팩코
792519                  현대자동차
827410              주식회사 쎄라테크
887840            (주)피오라뷰티코리아
910290               에스케이이(주)
917380             주식회사세광하이테크
930540               (주)톨리코리아
930572          이피에스에스글로벌주식회사
931306     애디언트오토모티브인테리어코리아(주
                  ...        
1140323               오리엔탈F&B
1142064             혜성음료 주식회사
1143061   

In [175]:
for i, v in enumerate(company_list):
    if '삼성' in v:
        print(i, v)

In [39]:
company_i = 2
cosine_sim = linear_kernel(test_matrix[company_i], tfidf_matrix)
print(test_name[company_i])

dk&d


In [40]:
sorted_index = sorted(range(len(cosine_sim[0])), key=lambda i: cosine_sim[0][i])[-100:]
sorted_index.reverse()
for i, v in enumerate(sorted_index):
    print(str(i), str(v), csv_company_series[v], str(cosine_sim[0][v]))

0 69325 주식회사 디케이앤디 (DK&D CO.LTD) 0.4857910369292537
1 68821 주식회사 디케이앤디(DK&D Co.Ltd) 0.4857910369292537
2 59802 주식회사 디케이앤디(DK&D CO.Ltd) 0.4857910369292537
3 46092 K&C 0.2529445494267581
4 4030 K&J 0.23562164160311044
5 48604 주식회사 DKS 0.1993078875100243
6 97659 DKR 0.19073495492699127
7 26947 DKCS 0.16034756490524588
8 83710 K&B상사 0.15988331991986116
9 69913 K & B상사 0.15988331991986116
10 53193 DK상사 0.15365185711802878
11 13329 DK섬유 0.14710257071660907
12 18538 동신E&D 0.14653898290149303
13 54742 DK동신주식회사 0.14363428710612208
14 9307 DK글로벌 0.14361452036587688
15 53789 신흥F&D 0.14264290404957086
16 37092 (유)성지F&D 0.1421697633855379
17 69694 (주)대도F&D 0.14195148542072344
18 14655 DK 강업  0.14087353496409583
19 71145 청해S&D 0.13969621625075196
20 69335 평화 M&D 0.13939114531895824
21 18201 가람D&D 0.13865147326419763
22 93943 한림 W&D 0.13792107972818643
23 81165 한림W&D 0.13792107972818643
24 54923 dk메디칼 0.13529022340477692
25 788 DK솔루션 0.13488729211184328
26 68653 엠제이R&D 0.13416354598019967
27 79581 K&

In [43]:
db_company_df = pd.read_sql_table(config['DB_TABLE_NAME_1'], con=conn, index_col='id', 
                          columns=['name'])
db_company_df.head()

,name
id,
1,롯데케미칼
2,대한유화
3,이화산업
4,디케이앤디
5,송원산업


In [45]:
company_series = db_company_df['name'].drop_duplicates().reset_index(drop=True)
company_series.head()

0    롯데케미칼
1     대한유화
2     이화산업
3    디케이앤디
4     송원산업
Name: name, dtype: object